In [9]:
import sys
import os

import numpy as np
from scipy import io

import matplotlib.pyplot as plt
%matplotlib notebook

import cvxpy

In [ ]:
"""#numTrain = 600
mat_dict = io.loadmat('port_reverse.mat') # TODO: REPLACE WITH test.mat FOR SUBMISSION

test_cl = mat_dict['test_cl']#[numTrain:]
test_hi = mat_dict['test_hi']#[numTrain:]
test_lo = mat_dict['test_lo']#[numTrain:]
test_op = mat_dict['test_op']#[numTrain:]
test_vol = mat_dict['test_vol']#[numTrain:]
numDays, numStocks = test_cl.shape

all_stocks = []
for i in range(numStocks):
    all_stocks.append(np.array_str(mat_dict['test_stocks'][0][i])[3:-2])"""

In [ ]:
"""mat_dict = io.loadmat('portfolio.mat')

train_cl = mat_dict['train_cl']#[:numTrain]
train_hi = mat_dict['train_hi']#[:numTrain]
train_lo = mat_dict['train_lo']#[:numTrain]
train_op = mat_dict['train_op']#[:numTrain]
train_vol = mat_dict['train_vol']#[:numTrain]
numTrain, _ = train_cl.shape

# make data compatible with our algorithms
train_cl = train_cl.T
train_op = train_op.T"""

In [10]:
numTrain = 750

mat_dict = io.loadmat('portfolio.mat')

test_cl = mat_dict['train_cl'][numTrain:]
test_hi = mat_dict['train_hi'][numTrain:]
test_lo = mat_dict['train_lo'][numTrain:]
test_op = mat_dict['train_op'][numTrain:]
test_vol = mat_dict['train_vol'][numTrain:]
numDays, numStocks = test_cl.shape

train_cl = mat_dict['train_cl'][:numTrain]
train_hi = mat_dict['train_hi'][:numTrain]
train_lo = mat_dict['train_lo'][:numTrain]
train_op = mat_dict['train_op'][:numTrain]
train_vol = mat_dict['train_vol'][:numTrain]

# make data compatible with our algorithms
train_cl = train_cl.T
train_op = train_op.T

all_stocks = []
for i in range(numStocks):
    all_stocks.append(np.array_str(mat_dict['train_stocks'][0][i])[3:-2])

In [3]:
def uniform_rebalance(t, portfolio_t, hi, lo, cl, op, stocks, vol):
    """% Input: 
    %%% t: we are at Day t
    %%% portfolio_t: current portfolio before open of Day t
    %%% hi: highest prices at Day t-1
    %%% lo: lowest prices at Day t-1
    %%% cl: close prices at Day t-1
    %%% op: open prices at Day t
    %%% stocks: names of all stocks
    %%% vol: volumes at Day t-1

    % Output: 
    %%% portfolio: a column vector, each entry represents the portion of 
    %%%            the value of the corresponding stock over the total cash 
    %%%            realizable value [as a result, the value of inactive stocks 
    %%%            i.e., with open price NaN, cannot be added into the total
    %%%            cash realizable value]. Hence, your output must satisfy
    %%%            sum(portfolio(isfinite(op))) < 1, and the rest is the 
    %%%            portion of cash realizable value you hold in cash. 
    %%% hold_flag: a binary vector representing whether you would like to hold
    %%%            (with value 1) each stock or you choose to rebalance it
    %%%            (with value 0). """
    
    numStocks = len(stocks)
    portfolio = np.zeros(numStocks)
    numActiveStock = np.sum(np.isfinite(op))
    portfolio[np.isfinite(op)] = 1.0/numActiveStock
        
    if np.mod(t,2) == 1:
        hold_flag = np.zeros(numStocks,dtype=bool)
    else:
        hold_flag = np.ones(numStocks,dtype=bool)      
    return portfolio, hold_flag

In [11]:
"""nan_stocks = np.unique(np.where(np.isnan(train_cl))[0])

k = 15
k_curr = 0
p_full = np.zeros([numStocks, numTrain])

# build p matrix for all stocks
for curr_t in range(1, numTrain):
    nan_locs = np.where(np.logical_or(np.isnan(train_cl[:, curr_t-1]),
               np.isnan(train_cl[:, curr_t])))[0]
    non_nan_locs = np.where(np.logical_and(np.isfinite(train_cl[:, curr_t-1]),
                   np.isfinite(train_cl[:, curr_t])))[0]
        
    p_full[nan_locs, curr_t] = 0
    p_full[non_nan_locs, curr_t] = (train_cl[non_nan_locs, curr_t]
        - train_cl[non_nan_locs, curr_t-1]) / train_cl[non_nan_locs, curr_t-1]
    
sorted_stocks = np.argsort(np.diag(np.cov(p_full)))
fundStocks = []

for i in sorted_stocks:
    if k_curr >= k:
        break
    if i not in nan_stocks:
        fundStocks.append(i)
        k_curr += 1

numFundStocks = k
#print np.sort(fundStocks)
#print sorted_stocks[:20]
#print nan_stocks"""

fsdax = [455, 47, 322, 216, 190, 269, 106, 389]
fscsx = [304, 197, 456, 279, 338, 112, 7, 227]
fbsox = [456, 279, 118, 6, 227, 179, 178]
fbiox = [84, 60, 192]
the_sahaana_special = [467, 457, 469, 486, 283, 221, 256, 34, 227, 187]

fundStocks = np.unique(fsdax + fscsx + fbsox + fbiox + the_sahaana_special)
numFundStocks = len(fundStocks)

In [23]:
# params
window_size = 450
update_interval = 50
mu = 1e2
eta = 1e0

# initialize initial p matrix
p_init = np.zeros([numFundStocks, numTrain])

# build initial p matrix
for curr_t in range(1, numTrain):
        
    nan_locs = np.where(np.logical_or(np.isnan(train_cl[fundStocks, curr_t-1]),
               np.isnan(train_cl[fundStocks, curr_t])))[0]
    non_nan_locs = np.where(np.logical_and(np.isfinite(train_cl[fundStocks, curr_t-1]),
                   np.isfinite(train_cl[fundStocks, curr_t])))[0]
        
    p_init[nan_locs, curr_t] = 0
    p_init[non_nan_locs, curr_t] = (train_cl[fundStocks,:][non_nan_locs, curr_t]
        - train_cl[fundStocks,:][non_nan_locs, curr_t-1]) / train_cl[fundStocks,:][non_nan_locs, curr_t-1]
    
# matrices to save data
cls = np.zeros([numStocks, numDays])
ops = np.zeros([numStocks, numDays])

def windowitz(t, portfolio_t, hi, lo, cl, op, stocks, vol):
    
    # save data
    cls[:, t] = cl.T
    ops[:, t] = op.T
    
    if t % update_interval is 0:
    
        # initialize p matrix
        p = np.zeros([numFundStocks, t])
    
        # build p matrix
        for curr_t in range(1, t):
            nan_locs = np.where(np.logical_or(np.isnan(cls[fundStocks, curr_t-1]),
                       np.isnan(cls[fundStocks, curr_t])))[0]
            #print nan_locs
            non_nan_locs = np.where(np.logical_and(np.isfinite(cls[fundStocks, curr_t-1]),
                           np.isfinite(cls[fundStocks, curr_t])))[0]
        
            p[nan_locs, curr_t] = 0
            p[non_nan_locs, curr_t] = (cls[fundStocks,:][non_nan_locs, curr_t]
                - cls[fundStocks,:][non_nan_locs, curr_t-1]) / cls[fundStocks,:][non_nan_locs, curr_t-1]
        
        # compute statistics
        pbar = np.mean(np.hstack([p_init, p])[:, t+numTrain-window_size:t+numTrain-1], axis=1)
        sigma = np.cov(np.hstack([p_init, p])[:, t+numTrain-window_size:t+numTrain-1])
        
        #for i in range(numFundStocks):
        #    for j in range(numFundStocks):
        #        if sigma[i, j] != sigma[j, i]:
        #            print fundStocks[i], fundStocks[j], sigma[i, j], sigma[j, i]
    
        # set up and solve markowitz problem
        b_opt = cvxpy.Variable(numFundStocks)
        objective = cvxpy.Minimize(-pbar*b_opt + mu*cvxpy.quad_form(b_opt, sigma)
                    + eta*cvxpy.pnorm(b_opt - portfolio_t[fundStocks], 2))
        constraints = [cvxpy.sum_entries(b_opt) >= 0.01, cvxpy.sum_entries(b_opt) <= 1.0]
        problem = cvxpy.Problem(objective, constraints)
        problem.solve(verbose=False, solver='ECOS')
                           
        b_opt = b_opt.value
        
        portfolio = np.zeros(numStocks)
        portfolio[fundStocks] = b_opt
        
        portfolio /= np.sum(portfolio)
    
        return portfolio, np.zeros(numStocks, dtype=bool)
    
    else:
        
        return portfolio_t, np.ones(numStocks, dtype=bool)

In [13]:
def rebalance(value_vec, value_realizable, portfolio_dst, cost_rate):
    """ % This function simulates the process of rebalancing. The key difficulty 
    % here is to compute the transaction cost, i.e., to solve the equation
    %
    % sum( cost_rate * abs( portfolio_dst .* (value_realizable - C) -
    % value_vec) ) = C
    % 
    % where C is the total transcation cost. We use the Banach contraction 
    % theorem to solve this equation, with convergence rate cost_rate^t for 
    % t-th iteration. 

    % Input: 
    %%% value_vec: current values of each active stock
    %%% value_realizable: total cash realizable value
    %%% portfolio_dst: the desired portfolio after rebalancing
    %%% cost_rate: ratio of transactions cost (0.0005 in our case)

    % Output:
    %%% new_value_vec: the value vector after rebalancing
    %%% trans_cost: the total transaction cost"""
    
    iter_num = 7
    trans_cost = 0
    for iter in range(iter_num):
        trans_cost = np.sum(cost_rate * np.abs(portfolio_dst * \
                        (value_realizable-trans_cost)-value_vec))
        
    new_value_vec = portfolio_dst * (value_realizable - trans_cost)
    return new_value_vec, trans_cost

In [ ]:
value_op_seq = np.zeros(numDays) # total value before open of each day
value_cl_seq = np.zeros(numDays) # total value before close of each day 
portfolio_seq = np.zeros((numStocks, numDays)) # portfolio before open of each day
last_close_price = np.NaN * np.ones(numStocks)
print numDays, numStocks

In [ ]:
# Initialization
op = test_op[0,:]
cl = np.NaN * np.ones(numStocks)
hi = np.NaN * np.ones(numStocks)
lo = np.NaN * np.ones(numStocks)
vol = np.NaN * np.ones(numStocks)
value_op_seq[0] = 1 # initially we have one unit of cash

In [ ]:
[10 ** i for i in [-2, -1, 0, 1, 2]]

In [24]:
eta = 0.01
mu = 30
window_size = 250
update_interval = 125

value_op_seq = np.zeros(numDays) # total value before open of each day
value_cl_seq = np.zeros(numDays) # total value before close of each day 
portfolio_seq = np.zeros((numStocks, numDays)) # portfolio before open of each day
last_close_price = np.NaN * np.ones(numStocks)

# Initialization
op = test_op[0,:]
cl = np.NaN * np.ones(numStocks)
hi = np.NaN * np.ones(numStocks)
lo = np.NaN * np.ones(numStocks)
vol = np.NaN * np.ones(numStocks)
value_op_seq[0] = 1 # initially we have one unit of cash

for t in range(numDays):

    # At the beginning of day t, we use all information up to the open 
    # price of day t to determine the capital allocation at the end of 
    # day t.

    # Your function should be placed here. Note that the input vector may
    # contain NaNs (at Day 1, or for some inactive stocks). 
    new_portfolio, hold_flag = windowitz(t, portfolio_seq[:,t], hi, lo, cl, op, all_stocks, vol)

    # Check whether your output is valid
    isActive = np.isfinite(op)
    noHold = np.logical_and(isActive, np.logical_not(hold_flag))
    if np.sum(new_portfolio[noHold]) > 1 + 1e-8:
        sys.stderr.write('The output portfolio is not valid! {}'.format(np.sum(new_portfolio[noHold])))

    # The value of our portfolio at the end of Day t before paying transaction costs
    cl = test_cl[t,:]
    value_vec = value_op_seq[t] * portfolio_seq[:,t]
    growth = cl[isActive] / last_close_price[isActive]-1
    growth[np.isnan(growth)] = 0
    revenue_vec = value_vec[isActive] * growth
    value_vec[isActive] = value_vec[isActive] + revenue_vec
    value_cl_seq[t] = value_op_seq[t] + np.sum(revenue_vec)


    # At the end of Day t, we use the close price of day t to adjust our 
    # portfolio to the desired percentage.
    if t <= numDays-2:
        nonActive = np.logical_not(isActive)
        value_realizable = value_cl_seq[t] - np.sum(value_vec[nonActive])
        new_value_vec, trans_cost = rebalance(value_vec[isActive], value_realizable, \
                                              new_portfolio[isActive], 0.0005)
        value_op_seq[t+1] = value_cl_seq[t] - trans_cost
        value_vec[isActive] = new_value_vec
        portfolio_seq[:,t+1] = value_vec / value_op_seq[t+1]

    last_close_price[isActive] = cl[isActive]

    # Update information
    if t <= numDays-2:
        op = test_op[t+1,:]
        hi = test_op[t,:]
        lo = test_lo[t,:]
        vol = test_vol[t,:]

return_seq = np.log(value_op_seq[1:] / value_op_seq[:-1])
sharpe = np.sqrt(252) * np.mean(return_seq) / np.std(return_seq)
print sharpe

/Users/rs/Dropbox/Workspace/portfolio-opt/venv/lib/python2.7/site-packages/ipykernel/__main__.py:69: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


2.03506191363


In [21]:
best_sharpe = -100
best_eta = None
best_mu = None
best_ws = None
best_ui = None

iter = 0

for eta in [10. ** i for i in [-2, -1.5, -1, -0.5, 0]]:
    for mu in [10. ** i for i in [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]]:
        for window_size in [25 * i for i in range(4, 20)]:
            for update_interval in [10 * i for i in range(1, 15)]:
                
                iter += 1
                if iter % 100 is 0:
                    print iter
                
                value_op_seq = np.zeros(numDays) # total value before open of each day
                value_cl_seq = np.zeros(numDays) # total value before close of each day 
                portfolio_seq = np.zeros((numStocks, numDays)) # portfolio before open of each day
                last_close_price = np.NaN * np.ones(numStocks)

                # Initialization
                op = test_op[0,:]
                cl = np.NaN * np.ones(numStocks)
                hi = np.NaN * np.ones(numStocks)
                lo = np.NaN * np.ones(numStocks)
                vol = np.NaN * np.ones(numStocks)
                value_op_seq[0] = 1 # initially we have one unit of cash

                for t in range(numDays):

                    # At the beginning of day t, we use all information up to the open 
                    # price of day t to determine the capital allocation at the end of 
                    # day t.

                    # Your function should be placed here. Note that the input vector may
                    # contain NaNs (at Day 1, or for some inactive stocks). 
                    new_portfolio, hold_flag = windowitz(t, portfolio_seq[:,t], hi, lo, cl, op, all_stocks, vol)

                    # Check whether your output is valid
                    isActive = np.isfinite(op)
                    noHold = np.logical_and(isActive, np.logical_not(hold_flag))
                    if np.sum(new_portfolio[noHold]) > 1 + 1e-8:
                        sys.stderr.write('The output portfolio is not valid! {}'.format(np.sum(new_portfolio[noHold])))

                    # The value of our portfolio at the end of Day t before paying transaction costs
                    cl = test_cl[t,:]
                    value_vec = value_op_seq[t] * portfolio_seq[:,t]
                    growth = cl[isActive] / last_close_price[isActive]-1
                    growth[np.isnan(growth)] = 0
                    revenue_vec = value_vec[isActive] * growth
                    value_vec[isActive] = value_vec[isActive] + revenue_vec
                    value_cl_seq[t] = value_op_seq[t] + np.sum(revenue_vec)


                    # At the end of Day t, we use the close price of day t to adjust our 
                    # portfolio to the desired percentage.
                    if t <= numDays-2:
                        nonActive = np.logical_not(isActive)
                        value_realizable = value_cl_seq[t] - np.sum(value_vec[nonActive])
                        new_value_vec, trans_cost = rebalance(value_vec[isActive], value_realizable, \
                                                              new_portfolio[isActive], 0.0005)
                        value_op_seq[t+1] = value_cl_seq[t] - trans_cost
                        value_vec[isActive] = new_value_vec
                        portfolio_seq[:,t+1] = value_vec / value_op_seq[t+1]

                    last_close_price[isActive] = cl[isActive]

                    # Update information
                    if t <= numDays-2:
                        op = test_op[t+1,:]
                        hi = test_op[t,:]
                        lo = test_lo[t,:]
                        vol = test_vol[t,:]

                return_seq = np.log(value_op_seq[1:] / value_op_seq[:-1])
                sharpe = np.sqrt(252) * np.mean(return_seq) / np.std(return_seq)
                if sharpe >= best_sharpe:
                    best_sharpe = sharpe
                    best_eta = eta
                    best_mu = mu
                    best_ws = window_size
                    best_ui = update_interval
                    print best_sharpe, best_eta, best_mu, best_ws, best_ui

print 'RESULTS'
print best_sharpe, best_eta, best_mu, best_ws, best_ui

/Users/rs/Dropbox/Workspace/portfolio-opt/venv/lib/python2.7/site-packages/ipykernel/__main__.py:69: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.
/Users/rs/Dropbox/Workspace/portfolio-opt/venv/lib/python2.7/site-packages/ipykernel/__main__.py:76: RuntimeWarning: invalid value encountered in log


1.57642884764 0.01 0.01 100 20
1.57642890818 0.01 0.01 100 40
1.74550568713 0.01 0.01 125 10
100
200
1.74563594314 0.01 0.0316227766017 125 10
300
400
1.7459873802 0.01 0.1 125 10
500
600
1.7470579677 0.01 0.316227766017 125 10
1.74705796842 0.01 0.316227766017 125 20
700
800
900
1.75000394755 0.01 1.0 125 10
1.83593629413 0.01 1.0 175 10
1000
1100
1.90340888889 0.01 3.16227766017 175 30
1200
1300
1.93649426907 0.01 10.0 125 90
1400
2.02619423498 0.01 10.0 250 90
1500
2.03005689065 0.01 31.6227766017 100 130
1600
2.03855742824 0.01 31.6227766017 175 90
2.07517629381 0.01 31.6227766017 250 120
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600


In [ ]:
fig = plt.figure()
plt.plot(value_op_seq)
plt.title('Portfolio Return over Time')
plt.ylabel('Return')
plt.xlabel('Time')
#fig.savefig('plots/k={}_eta={}_mu={}_ws={}_ui={}.pdf'.format(k, eta, mu, window_size, update_interval))
return_seq = np.log(value_op_seq[1:] / value_op_seq[:-1])
annual_return = 252 * np.mean(return_seq)
sharpe = np.sqrt(252) * np.mean(return_seq) / np.std(return_seq)
print annual_return, sharpe